Ideas:
train hugging face model on debate corpus

tag <--> concatted underlined portion of a paragraph in the card

In [1]:
from Vector_Utils import *
import re

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
prompt = "methane from fracking in the US alone guarantees existential warming"
document = Document("article.docx")
full_text = "\n".join([para.text for para in document.paragraphs])

In [3]:
prompt = llmd_prompt(prompt, full_text)

In [4]:
print("Prompt:", prompt)
emphasis_words = prompt.split()

Prompt: US fracking boom's methane emissions threaten global climate stability, undermining clean energy efforts.


In [5]:
vectorizer = vectorizer()
prompt_vec = vectorizer.encode(prompt)

In [7]:
paragraphs = document.paragraphs
sentence_enders = re.compile(r'(?<=[.!?])\s+')
para_split = 3
emphasis_sensitivity = 0.7
underline_proportion = 0.8
pruning_threshold = 0.2

for paragraph in paragraphs:
    highest = 0.4
    best = ()

    sentences = re.split(sentence_enders, paragraph.text)

    if not vectorizer.screen_paragraph(sentences, prompt_vec, threshold=0.4):
        make_small(paragraph)
        continue

    if len(sentences) > para_split:
        print("SPLIT")
        for i in range((len(sentences) // para_split) + 1):
            k = i * para_split
            print(k)
            if k + para_split <= len(sentences):
                sub_paragraph = " ".join(sentences[k:k+para_split])
            else:
                sub_paragraph = " ".join(sentences[k:])
            print(sub_paragraph)
            # if not vectorizer.screen_paragraph(sub_paragraph, prompt_vec, threshold=0.4):
            #     continue
            sub_highest = 0.4
            sub_best = ()
            clauses = clause_separator(sub_paragraph)
            print(clauses)
            for combination in greedy_combinations(clauses, round(len(clauses) * underline_proportion), vectorizer, prompt_vec, threshold=pruning_threshold):
                joined = " ".join(combination)
                joined = joined.replace("[SPLIT]", " ")
                vector = vectorizer.encode(joined)
                sim_score = vectorizer.similarity(vector, prompt_vec)
                if sim_score > sub_highest:
                    sub_highest = sim_score
                    combination = (clause.split("[SPLIT]") for clause in combination)
                    combination = (i for sub in combination for i in sub)
                    combination = tuple(combination)
                    sub_best = combination
            best += sub_best
    else:
        clauses = clause_separator(paragraph.text)
        print(clauses)
        for combination in greedy_combinations(clauses, round(len(clauses) * underline_proportion), vectorizer, prompt_vec, threshold=pruning_threshold):
            joined = " ".join(combination)
            joined = joined.replace("[SPLIT]", " ")
            vector = vectorizer.encode(joined)
            sim_score = vectorizer.similarity(vector, prompt_vec)
            if sim_score > highest:
                highest = sim_score
                combination = (clause.split("[SPLIT]") for clause in combination)
                combination = (i for sub in combination for i in sub)
                combination = tuple(combination)
                best = combination
    
    emphasis = [clause for clause in best if vectorizer.similarity(vectorizer.encode(clause), prompt_vec) > emphasis_sensitivity]
    print("output:")
    print(highest, best)
    underline_best_match_in_paragraph(paragraph, best, emphasis)

document.save("article_annotated.docx")

0.7508429288864136
['After a frightening study from last week showed that industrial methane emissions have been "vastly underestimated,', '" a new projection Friday that the United States is on track to become', 'the world\'s leading exporter of liquefied natural gas within five years provoked warnings that the American fracking boom could "end hope for climate stability.[SPLIT]"']
output:
0.7734031081199646 ('After a frightening study from last week showed that industrial methane emissions have been "vastly underestimated,', 'the world\'s leading exporter of liquefied natural gas within five years provoked warnings that the American fracking boom could "end hope for climate stability.', '"')
0.6196070313453674
SPLIT
0
Liquefied natural gas (LNG) is primarily composed of methane, a greenhouse gas that is 84 times more potent than carbon dioxide over a 20-year period. Methane emissions, by some estimates, are responsible for about a quarter of human-caused global warming. "Science conf